<a href="https://colab.research.google.com/github/fjadidi2001/Artificial_Intelligence_Learning/blob/master/final_insurance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Specify file path
file_path = '/content/drive/My Drive/telematics_syn.csv'

# Import pandas (assuming you want to use it to read the CSV)
import pandas as pd

# Read the CSV file
data = pd.read_csv(file_path)

# Print the DataFrame
print(data)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
       Duration  Insured.age Insured.sex  Car.age  Marital     Car.use  \
0           366           45        Male       -1  Married     Commute   
1           182           44      Female        3  Married     Commute   
2           184           48      Female        6  Married     Commute   
3           183           71        Male        6  Married     Private   
4           183           84        Male       10  Married     Private   
...         ...          ...         ...      ...      ...         ...   
99995       182           61        Male       10   Single     Private   
99996       192           48        Male        3  Married     Commute   
99997       184           50        Male        2   Single     Commute   
99998       184           76        Male        2  Married     Private   
99999       365           25      Female        2   Singl

In [ ]:
!pip install pytorch-tabnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 794.2 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [15]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [16]:
# Split data into features and target
X = data.drop('NB_Claim', axis=1)
y = data['NB_Claim']

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

print(f"Categorical columns: {categorical_cols}")
print(f"Numerical columns: {numerical_cols}")

# Preprocessing pipelines for numerical and categorical data
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Categorical columns: Index(['Insured.sex', 'Marital', 'Car.use', 'Region'], dtype='object')
Numerical columns: Index(['Duration', 'Insured.age', 'Car.age', 'Credit.score',
       'Annual.miles.drive', 'Years.noclaims', 'Territory',
       'Annual.pct.driven', 'Total.miles.driven', 'Pct.drive.mon',
       'Pct.drive.tue', 'Pct.drive.wed', 'Pct.drive.thr', 'Pct.drive.fri',
       'Pct.drive.sat', 'Pct.drive.sun', 'Pct.drive.2hrs', 'Pct.drive.3hrs',
       'Pct.drive.4hrs', 'Pct.drive.wkday', 'Pct.drive.wkend',
       'Pct.drive.rush am', 'Pct.drive.rush pm', 'Avgdays.week',
       'Accel.06miles', 'Accel.08miles', 'Accel.09miles', 'Accel.11miles',
       'Accel.12miles', 'Accel.14miles', 'Brake.06miles', 'Brake.08miles',
       'Brake.09miles', 'Brake.11miles', 'Brake.12miles', 'Brake.14miles',
       'Left.turn.intensity08', 'Left.turn.intensity09',
       'Left.turn.intensity10', 'Left.turn.intensity11',
       'Left.turn.intensity12', 'Right.turn.intensity08',
       'Right.turn.inten

In [17]:
# Define models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    'TabNet': TabNetClassifier()
}

# Function to evaluate models
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

# Initial testing with raw data
results = {}
for name, model in models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])
    accuracy = evaluate_model(pipeline, X_train, X_test, y_train, y_test)
    results[name] = accuracy

print("Initial Model Performance with Raw Data and Default Hyperparameters:")
print(results)


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-pac

epoch 0  | loss: 0.3634  |  0:00:06s
epoch 1  | loss: 0.15092 |  0:00:11s
epoch 2  | loss: 0.08578 |  0:00:17s
epoch 3  | loss: 0.0606  |  0:00:23s
epoch 4  | loss: 0.05035 |  0:00:28s
epoch 5  | loss: 0.04487 |  0:00:34s
epoch 6  | loss: 0.04638 |  0:00:39s
epoch 7  | loss: 0.04404 |  0:00:44s
epoch 8  | loss: 0.04389 |  0:00:51s
epoch 9  | loss: 0.04133 |  0:00:56s
epoch 10 | loss: 0.0408  |  0:01:01s
epoch 11 | loss: 0.03956 |  0:01:07s
epoch 12 | loss: 0.03845 |  0:01:12s
epoch 13 | loss: 0.03726 |  0:01:18s
epoch 14 | loss: 0.03717 |  0:01:23s
epoch 15 | loss: 0.03714 |  0:01:28s
epoch 16 | loss: 0.03615 |  0:01:35s
epoch 17 | loss: 0.03679 |  0:01:40s
epoch 18 | loss: 0.03708 |  0:01:45s
epoch 19 | loss: 0.03786 |  0:01:51s
epoch 20 | loss: 0.04159 |  0:01:56s
epoch 21 | loss: 0.04091 |  0:02:02s
epoch 22 | loss: 0.03782 |  0:02:07s
epoch 23 | loss: 0.03682 |  0:02:12s
epoch 24 | loss: 0.03604 |  0:02:18s
epoch 25 | loss: 0.03539 |  0:02:23s
epoch 26 | loss: 0.03566 |  0:02:29s
e

KeyboardInterrupt: 

# last step

## optimizing TabNet's performance and exploring new DL architectures for insurance applications.